# Weighted CDE estimation for a linear-Gaussian DAG

May 2024

## Preamble

In [1]:
# General importations.
import pandas as pd
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
import platform
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
plt.rcParams["figure.figsize"] = (4,4)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import networkx as nx
from econml.dml import DML, LinearDML, SparseLinearDML, CausalForestDML
from sklearn.linear_model import (Lasso, LassoCV, LogisticRegression,
                                  LogisticRegressionCV,LinearRegression,
                                  MultiTaskElasticNet,MultiTaskElasticNetCV)
from sklearn.ensemble import (RandomForestRegressor,RandomForestClassifier,
                              GradientBoostingRegressor,GradientBoostingClassifier)
from dowhy import CausalModel
import dowhy.datasets

# Custom scripts.
from ld3 import LD3
from ldp import LDP
from ldp_utils import LDPUtils
from effect_estimation import EstimationPipeline
from data_generation import DataGeneration

# View versioning.
print("python version     :", platform.python_version())
print("numpy version      :", np.__version__)
print("pandas version     :", pd.__version__)
print("matplotlib version :", matplotlib.__version__)
print("seaborn version    :", sns.__version__)

python version     : 3.10.13
numpy version      : 1.26.3
pandas version     : 2.1.4
matplotlib version : 3.8.0
seaborn version    : 0.12.2


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


## Define functions

In [3]:
def get_ATE(data: pd.DataFrame, 
            x: str, 
            y: str, 
            covar: list, 
            model: str = "linear") -> float:
    if model == "linear":
        reg = LinearRegression().fit(data[[x] + covar], data[y])
    elif model == "logistic":
        reg = LogisticRegression(penalty = None, max_iter = 500).fit(data[[x] + covar], data[y])
    ate = reg.coef_[0]
    return ate

In [4]:
def plot_nx(adjacency_matrix,
            labels,
            figsize = (10,10),
            dpi = 200,
            node_size = 800,
            arrow_size = 10):
    
    g = nx.from_numpy_array(adjacency_matrix, create_using = nx.DiGraph)
    plt.figure(figsize = figsize, dpi = dpi)  
    nx.draw_shell(g, 
                 node_size = node_size, 
                 labels = dict(zip(list(range(len(labels))), labels)), 
                 arrowsize = arrow_size,
                 node_color = "pink",
                 with_labels = True)
    plt.show()
    plt.close()

## Estimation

In [2]:
dg = DataGeneration()
ldputils = LDPUtils()

In [37]:
%%capture
replicates = 100
f1_list = []
precision_list = []
recall_list = []
sdc_list = []
cde_list = []
x_causes_y = 1
true_acde = ["Z1", "Z4b", "Z3c", "Z3d", "B2", "B3", "M2"]
for rep in range(replicates):
    df = dg.generate_linear_gaussian(n = 10000,
                                     x_causes_y = x_causes_y,
                                     xy_coeff = 1.25,
                                     coefficient_range = (0.75, 1.1))
    display(df.head())

    ld3 = LD3(data = df, 
              independence_test = "fisher")
    start = time.time()
    sdc,acde = ld3.get_sdc_cde_adjustment(exposure = "X", 
                                          outcome = "Y",
                                          alpha = 0.01)
    print("\nResults complete in {}s.".format(round(time.time() - start, 4)))
    print("Total independence tests performed:", ld3.total_tests)
    
    print("\nSDC =", sdc)
    if not sdc:
        print("No direct discrimination detected.")
    else:
        print("Direct discrimination detected.")
    print("\nTrue A_CDE      =", sorted(true_acde))
    print("Predicted A_CDE =", sorted(acde))
    cde = get_ATE(df, 
                  x = "X", 
                  y = "Y", 
                  covar = acde)
    print("\nPredicted weighted CDE:", cde)
    
    bool_true = [1 if x in true_acde else 0 for x in df.columns]
    bool_pred = [1 if x in acde else 0 for x in df.columns]
    f1 = f1_score(bool_true, bool_pred)
    precision = precision_score(bool_true, bool_pred)
    recall = recall_score(bool_true, bool_pred)
    f1_list.append(f1)
    precision_list.append(precision)
    recall_list.append(recall)
    sdc_list.append(sdc == x_causes_y)
    cde_list.append(cde)
    print("\nF1:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    print()
    
    df_results = pd.DataFrame({"Variable": ld3.pred_label_dict.keys(),
                               "Label": ld3.pred_label_dict.values()})
    display(df_results)

In [40]:
df.shape

In [38]:
print("SDC accuracy:", np.sum(sdc_list) / len(sdc_list))
mean, ci = ldputils.get_ci(cde_list)
print("Mean weighted CDE {} [{},{}] (variance = {}).".format(round(mean,2), round(ci[0],2), round(ci[1],2), np.var(cde_list)))

mean, ci = ldputils.get_ci(f1_list)
print("\nMean F1         = {} [{},{}]".format(round(mean,2), round(ci[0],2), round(ci[1],2)))
mean, ci = ldputils.get_ci(precision_list)
print("Mean precision  = {} [{},{}]".format(round(mean,2), round(ci[0],2), round(ci[1],2)))
mean, ci = ldputils.get_ci(recall_list)
print("Mean recall     = {} [{},{}]".format(round(mean,2), round(ci[0],2), round(ci[1],2)))

SDC accuracy: 1.0
Mean weighted CDE 1.25 [1.25,1.26] (variance = 0.0016820754149553868).

Mean F1         = 0.99 [0.99,1.0]
Mean precision  = 0.99 [0.98,1.0]
Mean recall     = 1.0 [1.0,1.0]
Mean F1         = 0.99 [0.99,1.0]


## End of document